# import

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.models import Model
from keras.utils import np_utils
import keras 
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


## '''read data and preprocess'''

In [5]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['MiddlePhalanxTW']
#for each in flist:
each = 'Adiac'

fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

In [6]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 176, 1, 1)
# x_test.shape:  (391, 176, 1, 1)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


# 类似VGG的卷积神经网络：

In [18]:
'''对比一下示例数据与所用数据的不同'''
x_ = np.random.random((10, 10, 10, 3))
y_ = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
print("# x_.shape: ",x_.shape)
print("# y_.shape: ",y_.shape)

# x_.shape:  (10, 10, 10, 3)
# y_.shape:  (100, 10)


array([[[[  4.89033496e-01,   8.87785390e-01,   8.01902942e-03],
         [  2.43661146e-01,   9.24611982e-02,   6.14745651e-01],
         [  8.56031750e-01,   9.38926254e-01,   5.69361455e-01],
         ..., 
         [  4.73535731e-01,   7.22420613e-01,   8.67720174e-01],
         [  1.08069015e-01,   5.33860024e-01,   2.15286169e-01],
         [  4.22824508e-01,   3.92013645e-01,   2.91933812e-01]],

        [[  6.81409662e-01,   8.72292199e-01,   2.05931954e-01],
         [  2.46154948e-01,   1.40822385e-01,   8.78539714e-01],
         [  3.19489163e-01,   6.42116469e-01,   3.30648718e-01],
         ..., 
         [  2.08748879e-01,   7.05215766e-01,   8.22010228e-01],
         [  5.25973012e-01,   1.32789936e-01,   4.92678853e-01],
         [  4.98027349e-01,   3.18630919e-01,   9.72358053e-02]],

        [[  6.52165385e-01,   6.50345148e-01,   5.50664296e-01],
         [  8.08081715e-01,   9.65816049e-01,   6.77025835e-01],
         [  8.33104566e-01,   4.65478167e-01,   4.750102

In [23]:
x_train.shape[1:]

(176, 1)

In [45]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

x = keras.layers.Input(x_train.shape[1:])
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.

# input: 391x176 images with 1 channels -> (176, 1, 1) tensors.
# this applies 32 convolution filters of size 3x3 each.


conv2 = keras.layers.Conv2D(128, 8, 1, border_mode='same')(x)
conv2 = keras.layers.normalization.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)
#conv2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
drop_out = Dropout(0.25)(conv2)

conv3 = keras.layers.Conv2D(256, 5, 1, border_mode='same')(conv2)
conv4 = keras.layers.Conv2D(128, 3, 1, border_mode='same')(conv3)
conv4 = keras.layers.normalization.BatchNormalization()(conv4)
conv4 = keras.layers.Activation('relu')(conv4)
#conv4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
drop_out = Dropout(0.25)(conv4)

conv4 = keras.layers.Flatten()(conv4)
conv4 = keras.layers.Dense(128, activation='softmax')(conv4)
drop_out = Dropout(0.25)(conv4)
out = keras.layers.Dense(nb_classes, activation='softmax')(conv4)

model = Model(input=x, output=out)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])


hist = model.fit(x_train, Y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, Y_test, batch_size=16)

# Print the testing results which has the lowest training loss.
#log = pd.DataFrame(hist.history)
#print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (8, 1), padding="same")`
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
390/390 [==============================] - 8s 19ms/step - loss: 3.6126 - acc: 0.0282
Epoch 2/10
390/390 [==============================] - 7s 17ms/step - loss: 3.6032 - acc: 0.0462
Epoch 3/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5948 - acc: 0.0410
Epoch 4/10
390/390 [==============================] - 6s 16ms/step - loss: 3.6037 - acc: 0.0308
Epoch 5/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5996 - acc: 0.0282
Epoch 6/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5966 - acc: 0.0308
Epoch 7/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5938 - acc: 0.0231
Epoch 8/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5912 - acc: 0.0333
Epoch 9/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5897 - acc: 0.0385
Epoch 10/10
391/391 [==============================] - 2s 6ms/step


# CNN

In [7]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# 全局变量
batch_size = batch_size
nb_classses = nb_classes
epochs = nb_epochs
# input image dimensions
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (3,1)
# convolution kernel size
kernel_size = (8,1)
kernel_size_2 = (3,1)
input_shape = (x_train.shape[1:])
print(input_shape)

(176, 1, 1)


In [8]:
#构建模型  1
model = Sequential()  
""" 
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        border_mode='same', 
                        input_shape=input_shape)) 
"""  
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),  
                        padding='same',  
                        input_shape=input_shape)) # 卷积层1  
model.add(Activation('relu')) #激活层  

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) #卷积层2  
model.add(Activation('relu')) #激活层

model.add(Convolution2D(128, (kernel_size_2[0], kernel_size_2[1]))) #卷积层3  
model.add(Activation('relu')) #激活层  


model.add(MaxPooling2D(pool_size=pool_size)) #池化层  
model.add(Dropout(0.25)) #神经元随机失活  
model.add(Flatten()) #拉成一维数据  
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  
model.add(Dropout(0.5)) #随机失活  
model.add(Dense(nb_classes)) #全连接层2  
model.add(Activation('softmax')) #Softmax评分  
  
#编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

In [9]:
#训练模型  
model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(x_test, Y_test))  
#评估模型  
score = model.evaluate(x_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 1s - loss: 3.6380 - acc: 0.0205 - val_loss: 3.6135 - val_acc: 0.0179
Epoch 2/2000
390/390 [==============================] - 1s - loss: 3.6016 - acc: 0.0436 - val_loss: 3.6077 - val_acc: 0.0358
Epoch 3/2000
390/390 [==============================] - 1s - loss: 3.6008 - acc: 0.0462 - val_loss: 3.6099 - val_acc: 0.0435
Epoch 4/2000
390/390 [==============================] - 1s - loss: 3.5820 - acc: 0.0385 - val_loss: 3.6055 - val_acc: 0.0972
Epoch 5/2000
390/390 [==============================] - 1s - loss: 3.5871 - acc: 0.0513 - val_loss: 3.5899 - val_acc: 0.0614
Epoch 6/2000
390/390 [==============================] - 1s - loss: 3.5599 - acc: 0.0769 - val_loss: 3.5858 - val_acc: 0.1074
Epoch 7/2000
390/390 [==============================] - 1s - loss: 3.5511 - acc: 0.0718 - val_loss: 3.5484 - val_acc: 0.0691
Epoch 8/2000
390/390 [==============================] - 1s - loss: 3.5139 - acc

390/390 [==============================] - 1s - loss: 2.2073 - acc: 0.3487 - val_loss: 2.0471 - val_acc: 0.4194
Epoch 66/2000
390/390 [==============================] - 0s - loss: 2.1362 - acc: 0.3821 - val_loss: 2.0691 - val_acc: 0.4118
Epoch 67/2000
390/390 [==============================] - 1s - loss: 2.1278 - acc: 0.3641 - val_loss: 2.0387 - val_acc: 0.4271
Epoch 68/2000
390/390 [==============================] - 0s - loss: 2.1007 - acc: 0.3590 - val_loss: 1.9747 - val_acc: 0.4399
Epoch 69/2000
390/390 [==============================] - 0s - loss: 2.0514 - acc: 0.4000 - val_loss: 2.1476 - val_acc: 0.3606
Epoch 70/2000
390/390 [==============================] - 0s - loss: 2.0911 - acc: 0.3692 - val_loss: 1.9266 - val_acc: 0.4962
Epoch 71/2000
390/390 [==============================] - 0s - loss: 2.1001 - acc: 0.3590 - val_loss: 1.9529 - val_acc: 0.4297
Epoch 72/2000
390/390 [==============================] - 0s - loss: 2.1186 - acc: 0.3744 - val_loss: 1.9183 - val_acc: 0.4425
Epoch 

390/390 [==============================] - 1s - loss: 1.4158 - acc: 0.5615 - val_loss: 1.3236 - val_acc: 0.5754
Epoch 130/2000
390/390 [==============================] - 1s - loss: 1.4459 - acc: 0.5282 - val_loss: 1.3247 - val_acc: 0.6113
Epoch 131/2000
390/390 [==============================] - 0s - loss: 1.3307 - acc: 0.5846 - val_loss: 1.3142 - val_acc: 0.5882
Epoch 132/2000
390/390 [==============================] - 0s - loss: 1.4028 - acc: 0.5462 - val_loss: 1.3409 - val_acc: 0.5473
Epoch 133/2000
390/390 [==============================] - 0s - loss: 1.2859 - acc: 0.5744 - val_loss: 1.2631 - val_acc: 0.6266
Epoch 134/2000
390/390 [==============================] - 0s - loss: 1.3776 - acc: 0.5051 - val_loss: 1.3078 - val_acc: 0.5703
Epoch 135/2000
390/390 [==============================] - 0s - loss: 1.3040 - acc: 0.5718 - val_loss: 1.2463 - val_acc: 0.6368
Epoch 136/2000
390/390 [==============================] - 0s - loss: 1.3947 - acc: 0.5487 - val_loss: 1.3008 - val_acc: 0.6215

390/390 [==============================] - 0s - loss: 1.0505 - acc: 0.6333 - val_loss: 1.0597 - val_acc: 0.7084
Epoch 194/2000
390/390 [==============================] - 0s - loss: 1.0435 - acc: 0.6410 - val_loss: 1.0659 - val_acc: 0.7212
Epoch 195/2000
390/390 [==============================] - 1s - loss: 1.0736 - acc: 0.6538 - val_loss: 1.0558 - val_acc: 0.7033
Epoch 196/2000
390/390 [==============================] - 0s - loss: 1.0485 - acc: 0.6308 - val_loss: 1.0496 - val_acc: 0.7187
Epoch 197/2000
390/390 [==============================] - 0s - loss: 0.9947 - acc: 0.6821 - val_loss: 1.0242 - val_acc: 0.7289
Epoch 198/2000
390/390 [==============================] - 0s - loss: 1.0016 - acc: 0.6718 - val_loss: 1.0337 - val_acc: 0.7263
Epoch 199/2000
390/390 [==============================] - 1s - loss: 1.0307 - acc: 0.6667 - val_loss: 1.1245 - val_acc: 0.6573
Epoch 200/2000
390/390 [==============================] - 0s - loss: 0.9270 - acc: 0.7000 - val_loss: 0.9819 - val_acc: 0.7340

390/390 [==============================] - 0s - loss: 0.8196 - acc: 0.7333 - val_loss: 0.9620 - val_acc: 0.7263
Epoch 258/2000
390/390 [==============================] - 1s - loss: 0.8134 - acc: 0.7179 - val_loss: 0.9729 - val_acc: 0.7366
Epoch 259/2000
390/390 [==============================] - 1s - loss: 0.7495 - acc: 0.7462 - val_loss: 0.9495 - val_acc: 0.7647
Epoch 260/2000
390/390 [==============================] - 1s - loss: 0.7835 - acc: 0.7436 - val_loss: 1.1012 - val_acc: 0.6829
Epoch 261/2000
390/390 [==============================] - 0s - loss: 0.8487 - acc: 0.7103 - val_loss: 1.0166 - val_acc: 0.7442
Epoch 262/2000
390/390 [==============================] - 1s - loss: 0.7231 - acc: 0.7487 - val_loss: 1.0642 - val_acc: 0.7187
Epoch 263/2000
390/390 [==============================] - 1s - loss: 0.8210 - acc: 0.7000 - val_loss: 0.9949 - val_acc: 0.7494
Epoch 264/2000
390/390 [==============================] - 1s - loss: 0.7668 - acc: 0.7590 - val_loss: 0.9808 - val_acc: 0.7340

...

390/390 [==============================] - 0s - loss: 0.1008 - acc: 0.9667 - val_loss: 1.3757 - val_acc: 0.8005
Epoch 1730/2000
390/390 [==============================] - 0s - loss: 0.1128 - acc: 0.9590 - val_loss: 1.5054 - val_acc: 0.7775
Epoch 1731/2000
390/390 [==============================] - 0s - loss: 0.0977 - acc: 0.9769 - val_loss: 1.4118 - val_acc: 0.7775
Epoch 1732/2000
390/390 [==============================] - 0s - loss: 0.1021 - acc: 0.9641 - val_loss: 1.4004 - val_acc: 0.7928
Epoch 1733/2000
390/390 [==============================] - 0s - loss: 0.1329 - acc: 0.9436 - val_loss: 1.4888 - val_acc: 0.7954
Epoch 1734/2000
390/390 [==============================] - 0s - loss: 0.1057 - acc: 0.9692 - val_loss: 1.4618 - val_acc: 0.7903
Epoch 1735/2000
390/390 [==============================] - 0s - loss: 0.1035 - acc: 0.9641 - val_loss: 1.4863 - val_acc: 0.7826
Epoch 1736/2000
390/390 [==============================] - 0s - loss: 0.1158 - acc: 0.9590 - val_loss: 1.4226 - val_acc:

390/390 [==============================] - 0s - loss: 0.1129 - acc: 0.9615 - val_loss: 1.3999 - val_acc: 0.7954
Epoch 1794/2000
390/390 [==============================] - 0s - loss: 0.1209 - acc: 0.9513 - val_loss: 1.4015 - val_acc: 0.8082
Epoch 1795/2000
390/390 [==============================] - 0s - loss: 0.0529 - acc: 0.9795 - val_loss: 1.4549 - val_acc: 0.8056
Epoch 1796/2000
390/390 [==============================] - 0s - loss: 0.0995 - acc: 0.9641 - val_loss: 1.4222 - val_acc: 0.8235
Epoch 1797/2000
390/390 [==============================] - 0s - loss: 0.0953 - acc: 0.9744 - val_loss: 1.4569 - val_acc: 0.7980
Epoch 1798/2000
390/390 [==============================] - 0s - loss: 0.0986 - acc: 0.9641 - val_loss: 1.4151 - val_acc: 0.7980
Epoch 1799/2000
390/390 [==============================] - 0s - loss: 0.1306 - acc: 0.9462 - val_loss: 1.4830 - val_acc: 0.8005
Epoch 1800/2000
390/390 [==============================] - 0s - loss: 0.1012 - acc: 0.9590 - val_loss: 1.4477 - val_acc:

390/390 [==============================] - 0s - loss: 0.1490 - acc: 0.9487 - val_loss: 1.3753 - val_acc: 0.8056
Epoch 1858/2000
390/390 [==============================] - 0s - loss: 0.0671 - acc: 0.9769 - val_loss: 1.3391 - val_acc: 0.8056
Epoch 1859/2000
390/390 [==============================] - 0s - loss: 0.0894 - acc: 0.9590 - val_loss: 1.4770 - val_acc: 0.8082
Epoch 1860/2000
390/390 [==============================] - 0s - loss: 0.1164 - acc: 0.9590 - val_loss: 1.4116 - val_acc: 0.7980
Epoch 1861/2000
390/390 [==============================] - 0s - loss: 0.1008 - acc: 0.9487 - val_loss: 1.3541 - val_acc: 0.7980
Epoch 1862/2000
390/390 [==============================] - 1s - loss: 0.0794 - acc: 0.9692 - val_loss: 1.3576 - val_acc: 0.8005
Epoch 1863/2000
390/390 [==============================] - 0s - loss: 0.0685 - acc: 0.9769 - val_loss: 1.3929 - val_acc: 0.8005
Epoch 1864/2000
390/390 [==============================] - 0s - loss: 0.0977 - acc: 0.9667 - val_loss: 1.4137 - val_acc:

390/390 [==============================] - 0s - loss: 0.1072 - acc: 0.9641 - val_loss: 1.4825 - val_acc: 0.7877
Epoch 1922/2000
390/390 [==============================] - 0s - loss: 0.1221 - acc: 0.9615 - val_loss: 1.3884 - val_acc: 0.7954
Epoch 1923/2000
390/390 [==============================] - 0s - loss: 0.0828 - acc: 0.9795 - val_loss: 1.4802 - val_acc: 0.7954
Epoch 1924/2000
390/390 [==============================] - 0s - loss: 0.0618 - acc: 0.9821 - val_loss: 1.4661 - val_acc: 0.7954
Epoch 1925/2000
390/390 [==============================] - 0s - loss: 0.0704 - acc: 0.9744 - val_loss: 1.4437 - val_acc: 0.7877
Epoch 1926/2000
390/390 [==============================] - 0s - loss: 0.0973 - acc: 0.9744 - val_loss: 1.4817 - val_acc: 0.7928
Epoch 1927/2000
390/390 [==============================] - 0s - loss: 0.0762 - acc: 0.9744 - val_loss: 1.4106 - val_acc: 0.8005
Epoch 1928/2000
390/390 [==============================] - 0s - loss: 0.1155 - acc: 0.9564 - val_loss: 1.4373 - val_acc:

390/390 [==============================] - 0s - loss: 0.1113 - acc: 0.9564 - val_loss: 1.5375 - val_acc: 0.8005
Epoch 1986/2000
390/390 [==============================] - 0s - loss: 0.0951 - acc: 0.9538 - val_loss: 1.4601 - val_acc: 0.7903
Epoch 1987/2000
390/390 [==============================] - 1s - loss: 0.0993 - acc: 0.9692 - val_loss: 1.5193 - val_acc: 0.8005
Epoch 1988/2000
390/390 [==============================] - 0s - loss: 0.1750 - acc: 0.9462 - val_loss: 1.4681 - val_acc: 0.8005
Epoch 1989/2000
390/390 [==============================] - 0s - loss: 0.1295 - acc: 0.9564 - val_loss: 1.4841 - val_acc: 0.7928
Epoch 1990/2000
390/390 [==============================] - 0s - loss: 0.0809 - acc: 0.9744 - val_loss: 1.5228 - val_acc: 0.7877
Epoch 1991/2000
390/390 [==============================] - 0s - loss: 0.0963 - acc: 0.9564 - val_loss: 1.5031 - val_acc: 0.8031
Epoch 1992/2000
390/390 [==============================] - 0s - loss: 0.0447 - acc: 0.9846 - val_loss: 1.5168 - val_acc:

In [15]:
#构建模型  2
model = Sequential()  
""" 
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        border_mode='same', 
                        input_shape=input_shape)) 
"""  
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),  
                        padding='same',  
                        input_shape=input_shape)) # 卷积层1  
model.add(Activation('relu')) #激活层  
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) #卷积层2  
model.add(Activation('relu')) #激活层  
model.add(MaxPooling2D(pool_size=pool_size)) #池化层  
model.add(Dropout(0.25)) #神经元随机失活  
model.add(Flatten()) #拉成一维数据  
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  
model.add(Dropout(0.5)) #随机失活  
model.add(Dense(nb_classes)) #全连接层2  
model.add(Activation('softmax')) #Softmax评分  
  
#编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

In [18]:
#训练模型  
model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(x_test, Y_test))  
#评估模型  
score = model.evaluate(x_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

Train on 390 samples, validate on 391 samples
Epoch 1/50
390/390 [==============================] - 2s 6ms/step - loss: 3.6304 - acc: 0.0231 - val_loss: 3.6197 - val_acc: 0.0256
Epoch 2/50
390/390 [==============================] - 2s 5ms/step - loss: 3.6076 - acc: 0.0205 - val_loss: 3.6129 - val_acc: 0.0179
Epoch 3/50
390/390 [==============================] - 2s 5ms/step - loss: 3.6021 - acc: 0.0333 - val_loss: 3.6260 - val_acc: 0.0153
Epoch 4/50
390/390 [==============================] - 2s 5ms/step - loss: 3.6041 - acc: 0.0333 - val_loss: 3.6112 - val_acc: 0.0358
Epoch 5/50
390/390 [==============================] - 2s 5ms/step - loss: 3.5983 - acc: 0.0564 - val_loss: 3.6187 - val_acc: 0.0435
Epoch 6/50
390/390 [==============================] - 2s 4ms/step - loss: 3.5629 - acc: 0.0718 - val_loss: 3.5719 - val_acc: 0.0921
Epoch 7/50
390/390 [==============================] - 2s 4ms/step - loss: 3.5492 - acc: 0.0821 - val_loss: 3.5402 - val_acc: 0.1125
Epoch 8/50
390/390 [==========